In [48]:
import cv2
import os
from os import listdir
from os.path import isfile, join
from PIL import Image as Image
import numpy as np

from scipy.optimize import curve_fit

def relit(x, a, b):
    return (a * x.astype(np.float)/255 + b)*255



from matplotlib import pyplot as plt
def plshow(im,title='MINE'):
    if len(im.shape)>2:
  #      plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
        plt.imshow(im)
    else:
        plt.imshow(im,cmap='gray')
    plt.title(title)
    plt.rcParams["figure.figsize"] = (80,12)
    plt.show()

sd_path = 'dataset/ISTD/train_A'
mask_path = 'dataset/ISTD/train_B'
sdfree_path = 'dataset/ISTD/train_C_fixed_ours'

out = 'dataset/ISTD/train_params/'
if not os.path.exists(out):
    os.makedirs(out)

im_list  =  [f for f in listdir(sd_path) if isfile(join(sd_path, f)) and f.endswith('png')]
print(len(im_list),im_list[0])
kernel = np.ones((5,5),np.uint8)


def im_relit(Rpopt,Gpopt,Bpopt,dump):
    #fc this shit
    sdim = dump.copy()
    sdim.setflags(write=1)
    sdim = sdim.astype(np.float)
    sdim[:,:,0] = (sdim[:,:,0]/255) * Rpopt[0] + Rpopt[1]
    sdim[:,:,1] = (sdim[:,:,1]/255) * Gpopt[0] + Gpopt[1]
    sdim[:,:,2] = (sdim[:,:,2]/255) * Bpopt[0] + Bpopt[1]
    sdim = sdim*255
   # print(np.amin(sdim),np.amax(sdim))
    return sdim

errors= []
for im in im_list:
    sd = np.asarray(Image.open(join(sd_path,im)))
    mean_sdim = np.mean(sd,axis=2)
    
    mask_ori = np.asarray(Image.open(join(mask_path,im)))
    mask = cv2.erode(mask_ori ,kernel,iterations = 2)

    
    sdfree = np.asarray(Image.open(join(sdfree_path,im)))
    mean_sdfreeim = np.mean(sdfree,axis=2)
    
    #pixels for regression funtion
    i, j = np.where(np.logical_and(np.logical_and(np.logical_and(mask>=1,mean_sdim>5),mean_sdfreeim<230),np.abs(mean_sdim-mean_sdfreeim)>10))

    source = sd*0
    source[tuple([i,j])] = sd[tuple([i,j])] 
    target = sd*0
    target[tuple([i,j])]= sdfree[tuple([i,j])]
    
    R_s = source[:,:,0][tuple([i,j])]
    G_s = source[:,:,1][tuple([i,j])]
    B_s = source[:,:,2][tuple([i,j])]
    
    R_t = target[:,:,0][tuple([i,j])]
    G_t = target[:,:,1][tuple([i,j])]
    B_t = target[:,:,2][tuple([i,j])]
    
    c_bounds = [[1,-0.1],[10,0.5]]

    
    Rpopt, pcov = curve_fit(relit, R_s, R_t,bounds=c_bounds)
    Gpopt, pcov = curve_fit(relit, G_s, G_t,bounds=c_bounds)
    Bpopt, pcov = curve_fit(relit, B_s, B_t,bounds=c_bounds)
    
    
    relitim = im_relit(Rpopt,Gpopt,Bpopt,sd)
    
    #final = sd.copy()
    #final[tuple([i,j])] = relitim[tuple([i,j])]
    #final[final>255] =255
    #final[final<0] = 0

    #plshow(final)
    error = np.mean(np.abs(relitim[tuple([i,j])].astype(np.float) - sdfree[tuple([i,j])]).astype(np.float))
    print(error,Rpopt,Gpopt,Bpopt)
    f = open(join(out,im+'.txt'),"a")
    f.write("%f %f %f %f %f %f"%(Rpopt[1],Rpopt[0],Gpopt[1],Gpopt[0],Bpopt[1],Bpopt[0]))
    f.close()
             
  #  print(error)
    errors.append(error)
                    
    
print(np.mean(errors))
#no bound - 8.55
#### y_bound ###error
#    0.5        8.86
#    0.1        15.692271753155671    
#    0.25       10.830443545867785
#    1          8.86

1330 13-14.png
5.009201165757417 [ 2.4577046 -0.0107077] [1.96292375 0.03742387] [1.66641561 0.05142359]
12.557166108231598 [1.         0.47089183] [1.01141423 0.40761759] [1.         0.35819274]
7.532163787052489 [1.36470302 0.11617836] [1.35946438 0.07884616] [1.2162239  0.09544635]
19.61040195664874 [1.         0.46429342] [1.         0.41372481] [1.         0.35411328]
12.387102593381845 [1.         0.40474525] [1.         0.36618001] [1.         0.30457438]
6.241485039361433 [1.24169934 0.24815966] [1.20049734 0.21402725] [1.20519596 0.12513773]
5.785971619706131 [1.51092218 0.34851648] [1.21188995 0.34101258] [1.02058334 0.29614014]
17.118862751395312 [1.66188257 0.26537164] [1.35520425 0.28154068] [1.77398978 0.11631677]
4.218141292961971 [1.13025247 0.38224351] [1.10667088 0.33890229] [1.         0.31539165]
11.894605985113497 [1.3901423  0.18765527] [1.35044863 0.16179696] [1.15484409 0.16976616]
11.361911276969092 [1.36932553 0.20350377] [1.18084095 0.20083562] [1.14959475 0.

9.132013795788252 [1.         0.34806853] [1.         0.31731916] [1.         0.27023051]
9.792317679752305 [1.         0.36588639] [1.       0.314011] [1.         0.25793671]
5.940965701437201 [1.         0.20803771] [1.         0.17323618] [1.         0.13119351]
9.882733508141538 [1.30672327 0.19066968] [1.11155731 0.21128187] [1.1492221  0.10612051]
10.405425120944802 [1.         0.43826299] [1.         0.39470588] [1.         0.34912023]
7.060083238993572 [1.3451903  0.17801084] [1.27648049 0.16115865] [1.19106949 0.16273088]
7.1223147442726695 [1.11513652 0.36908961] [1.55617714 0.18893727] [1.23582137 0.15631021]
9.293580082372218 [1.         0.32047473] [1.        0.2833482] [1.         0.23739715]
8.005524059669257 [1.28987369 0.21332551] [1.25316586 0.18238389] [1.20759584 0.11579795]
7.76911158735872 [1.21623172 0.20625933] [1.18067534 0.18222335] [1.16642815 0.10963884]
5.1918998443205195 [1.66332463 0.41683789] [1.39129656 0.3901788 ] [1.11576509 0.35980625]
10.69094639243

8.229391799374683 [1.11326697 0.28303568] [1.22700416 0.18165858] [1.18957228 0.16675759]
5.817884163170584 [1.01720958 0.32880787] [1.02453528 0.30305116] [1.00257847 0.26936701]
11.327098166250956 [1.03155823 0.25565992] [1.05183265 0.21572989] [1.02914384 0.1852006 ]
7.149110333966241 [1.1455228  0.21281691] [1.13961522 0.20255734] [1.13505086 0.16736176]
10.3813820806001 [1.         0.38640003] [1.         0.32192303] [1.02079451 0.23596475]
10.112401221394446 [1.         0.37742396] [1.15628001 0.27022072] [1.06136077 0.19953287]
8.233764537328963 [1.13845694 0.24339034] [1.13403178 0.2055601 ] [1.06929967 0.18204631]
7.166512160354089 [1.13639967 0.20977336] [1.13292441 0.19817585] [1.13138368 0.16510997]
9.43711205936275 [1.         0.33920725] [1.19067778 0.18427091] [1.17262074 0.15875156]
5.7627867405678685 [1.         0.33192273] [1.         0.30700443] [1.         0.26494425]
5.799446886327617 [1.55970436 0.28742759] [1.28359163 0.27651333] [1.26502914 0.17683042]
10.664803

12.92374294802015 [1.         0.43825739] [1.         0.40630202] [1.         0.35982225]
12.12330184772465 [1.         0.31878988] [1.         0.29516628] [1.         0.26243926]
13.02981427411799 [1.22953443 0.27054121] [1.13130629 0.25406708] [1.02533223 0.21277484]
11.11366702434372 [1.0001778  0.15644102] [1.        0.1407532] [1.07866234 0.05775561]
13.643759331445702 [2.14823332 0.33184356] [2.03092151 0.29867382] [1.92904248 0.22367221]
7.744886110161616 [1.20591221 0.45744155] [1.24288846 0.39719454] [1.08658366 0.36137132]
10.809995204901764 [1.18865417 0.15685477] [1.08331153 0.15662978] [1.14261109 0.08033617]
3.7238308588415934 [1.00328037 0.20303921] [1.00661734 0.18357931] [1.         0.15610408]
7.936937679628445 [1.07265777 0.29389113] [1.06746621 0.25758217] [1.09819489 0.17303536]
4.283265055990391 [1.        0.1119575] [1.         0.08966731] [1.         0.06296653]
6.534887905244455 [1.41717386 0.15807677] [1.33269811 0.14386502] [1.15669023 0.11879725]
5.944232542

10.399040863387082 [1.         0.34256646] [1.         0.31658962] [1.        0.2745839]
15.365551767647597 [1.        0.3479788] [1.06557821 0.29742744] [1.05550735 0.2351434 ]
5.530280188449658 [1.32775307 0.23470935] [1.26303032 0.22132559] [1.14345919 0.20878732]
8.872598365341643 [1.         0.29197827] [1.         0.25541775] [1.         0.20189065]
4.363754424405304 [1.08907136 0.26384361] [1.04647512 0.24163538] [1.         0.21388896]
13.529900174983718 [1.         0.41353479] [1.        0.4040192] [1.         0.38084503]
6.579937201837601 [1.09192602 0.35805918] [1.11899994 0.30419861] [1.         0.29325943]
11.521328965457522 [1.         0.32320311] [1.         0.30077645] [1.         0.25973495]
9.514036722453081 [1.         0.14215642] [1.         0.12358407] [1.01925728 0.08379428]
13.70804099395271 [1.         0.39063537] [1.         0.36265178] [1.         0.31751861]
11.592487347281375 [1.77474634 0.41289645] [1.42922063 0.41914452] [1.25133301 0.3907739 ]
4.588247546

...

5.581341597426848 [1.        0.2900611] [1.         0.24783373] [1.         0.18492683]
5.311604781287308 [1.08782496 0.09834222] [1.06928917 0.0586435 ] [1.10309036 0.04706468]
11.947792958577935 [1.18489773 0.22567682] [1.17241542 0.21127795] [1.14331286 0.18418474]
7.245979577607476 [1.         0.27493374] [1.         0.23770554] [1.         0.18181668]
9.495213434127194 [1.44451906 0.16169426] [1.39969108 0.15862781] [1.33265799 0.13646104]
11.979729601337414 [1.         0.41507488] [1.         0.38521274] [1.         0.34710955]
5.2193883889384045 [1.         0.39587723] [1.         0.37685163] [1.         0.33322685]
5.744075709511627 [1.02852747 0.23553349] [1.02116077 0.21611541] [1.00459175 0.19130408]
5.23308406819318 [1.4429346  0.39336271] [1.20736696 0.37503816] [1.07921141 0.32431613]
4.722516315006456 [1.3978819  0.08579017] [1.37456425 0.04520417] [1.2666831  0.04766728]
13.911817208795442 [1.         0.39398167] [1.         0.36708687] [1.         0.32054315]
6.4613818

4.093994707301205 [1.08729292 0.08806179] [1.06692303 0.06636428] [1.03433912 0.05085375]
4.803442691616833 [1.         0.49318999] [1.         0.45994491] [1.         0.41414422]
8.141877447013025 [1.       0.378827] [1.         0.35032184] [1.         0.31377014]
5.469339275345639 [1.05194881 0.49235425] [1.        0.4531681] [1.         0.36628546]
10.552513200233147 [1.         0.46403819] [1.02642136 0.4204548 ] [1.00651206 0.36114381]
7.912086669941295 [1.4773809  0.02540787] [1.         0.24006762] [1.         0.22141444]
5.1324676655653985 [1.04788493 0.39280235] [1.         0.36007542] [1.         0.29963288]
8.464687671158348 [1.         0.41448182] [1.         0.37280512] [1.         0.30366964]
8.831959464848165 [1.01165928 0.38713134] [1.19958591 0.27802433] [1.         0.27581693]
3.689023065396748 [1.23016991 0.19426718] [1.        0.2285476] [1.         0.19498156]
7.739213941895897 [1.14601354 0.24850415] [1.13882119 0.21963422] [1.0931114 0.1761189]
4.760667671639348 

6.0598566365207 [1.47693262 0.29409661] [1.28154119 0.2810116 ] [1.17061817 0.24014481]
6.779243867416825 [1.45616599 0.02661183] [1.1304123  0.16180863] [1.         0.22385116]
14.01559868222701 [1.         0.47228961] [1.         0.43049894] [1.         0.37735332]
8.385329599986996 [1.17041771 0.24841201] [1.14274271 0.22358384] [1.09631354 0.17970203]
16.0227451383004 [1.         0.34002811] [1.         0.32543527] [1.50375927 0.12814954]
6.535906025140597 [1.08511613 0.13623745] [1.0503657  0.12126746] [1.12643462 0.03720826]
5.605220318668054 [1.48784368 0.09234066] [1.40130347 0.08766213] [1.27551035 0.06062238]
9.740896512046973 [1.43057947 0.16835786] [1.18733111 0.19233386] [1.18582144 0.09265421]
24.62920484905838 [1.19588232 0.39341377] [1.30426752 0.33270039] [1.31818841 0.24892338]
9.87361167273439 [1.         0.35505983] [1.03153041 0.28011414] [1.         0.25790464]
10.785432130473906 [1.0100719  0.22856359] [1.         0.20751649] [1.         0.17314927]
2.84882287021

3.7685768629199634 [1.         0.25506584] [1.         0.24477779] [1.         0.22149523]
12.170617812549592 [1.         0.43683909] [1.         0.40373471] [1.         0.34853848]
12.144325023976737 [1.16927628 0.29559312] [1.09902816 0.270262  ] [1.02206552 0.22197826]
7.8854542794127775 [1.07043808 0.4451352 ] [1.         0.41019962] [1.         0.34149892]
5.747458873321492 [1.04981411 0.24164911] [1.07435791 0.20507094] [1.11552737 0.13974482]
4.859785336888636 [1.32823536 0.35237545] [1.17587717 0.31881741] [1.13394924 0.24219468]
6.1713753736067 [1.14537152 0.22041008] [1.12506424 0.20464841] [1.0743455  0.18970252]
6.389784160204872 [1.00705619 0.12245689] [1.         0.10373429] [1.05016515 0.05450275]
4.843863878027568 [1.54593086 0.06172297] [1.4261355  0.06443288] [1.28244067 0.06144386]
9.944508788523496 [1.         0.43965845] [1.         0.40195183] [1.         0.34397386]
7.146629277531506 [1.08914226 0.18917886] [1.0781795  0.15343313] [1.04779075 0.11422093]
6.251557

4.608339372745333 [1.74113426 0.34798842] [1.52317166 0.3191022 ] [1.16724291 0.2942582 ]
18.875741071133284 [1.2543283  0.15648117] [1.06798541 0.18717109] [1.34530144 0.04991616]
11.148313150594289 [1.         0.37511565] [1.09509921 0.28287656] [1.14942929 0.17369543]
7.233035561460236 [1.08561326 0.2354706 ] [1.20422952 0.14769201] [1.1853493  0.09605807]
8.863229059881018 [1.41297349 0.19649819] [1.33371269 0.18662474] [1.22792143 0.15976778]
19.45745868435802 [1.49994807 0.26682565] [1.26711735 0.27545718] [1.34188963 0.14555973]
9.468170012731786 [1.         0.42522692] [1.         0.38323518] [1.         0.32367061]
5.868515168416221 [1.52897243 0.06848402] [1.38364141 0.07468658] [1.25120255 0.06850521]
6.000131214338976 [1.         0.12002132] [1.        0.0980278] [1.0169329  0.05973255]
7.438143572576681 [1.         0.28502836] [1.         0.26303692] [1.         0.22860402]
5.289794627964023 [1.1335567  0.21179734] [1.11170322 0.19507687] [1.07163701 0.16266489]
7.96468077